# Game Generation Research Notebook

The purpose of this notebook is to provide an overview of my research into game theory, specifically in the application of LLMs for dynamically generating games. This research project will focus on sequential games, though other games may be explored later.

In [1]:
import requests
from typing import Tuple
import json

def get_service() -> Tuple[str, str]:
        """
        Loads the credentials for the service.

        Returns:
            token: token to use for authentication.
            svc_url: url of the service instance.
        """
        credentials_path = "./credentials.json"
        try:
            with open(credentials_path, "r") as file:
                svc_key = json.loads(file.read())
        except Exception as error:
            print(f"Could not load the credentials: {error}")

        return requests.post(
            f"{svc_key['uaa']['url']}/oauth/token",
            auth=(svc_key["uaa"]["clientid"], svc_key["uaa"]["clientsecret"]),
            params={"grant_type": "client_credentials"}
        ).json()["access_token"], svc_key["url"]

def prompt_llm(prior, prompt, contexts=[], temp=0.7) -> str:
        """
        Get the completions from the LLM.
        """
        if prompt == "": return ""
        token, svc_url = get_service()
        edited_prompt = prompt
        messages = [{'role': 'assistant', 'content': context} for context in contexts]
        messages.append({"role": "user", "content": edited_prompt})
        messages.insert(0, {"role": "system", "content": prior})
        response = requests.post(
            f"{svc_url}/api/v1/completions",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            
            json={
                "deployment_id": "gpt-4-32k",
                "messages": messages,
                "max_tokens": 500,
                "temperature": temp,
                "frequency_penalty": 0,
                "presence_penalty": 0,
                "top_p": 0.95,
                "stop": "null"
            }
        )

        if response.status_code != 200:
            print(f"Error: {response.text}")
            return False, ""
        return str(response.json()["choices"][0]["message"]["content"])

## Dynamic Code Generation & Execution

The following is a demonstration of how to dynamically execute code in Python. This will be useful for generating sequential games.

In [2]:
def extract_code(response):
    code = response.split("```")
    if len(code) > 1:
        return code[1][len("python"):]
    return response

def generate_code(prompt, contexts=[], temp=0.2):
    response = prompt_llm(
        prior="You are a coding assistant. You are helping a user write a Python program. Only respond with relevant code; no explanation or comments.",
        prompt=prompt,
        contexts=contexts,
        temp=temp
    )
    return extract_code(response)

In [3]:
code = generate_code("Write code for a Python program that prints the first 10 twin prime numbers.")
print(code)

def twin_primes(n):
    primes = []
    i = 2
    while len(primes) < n:
        if all(i % p != 0 for p in primes):
            if primes and i - primes[-1] == 2:
                print((primes[-1], i))
            primes.append(i)
        i += 1

twin_primes(10)


In [4]:
exec(code)

(3, 5)
(5, 7)
(11, 13)
(17, 19)


## Sequential Games

Sequential games are a type of game where players take turns to make decisions. The decisions made by each player can be based on the decisions made by the previous players. Sequential games are often represented as trees, where the nodes represent decision points and the edges represent the possible decisions that can be made.

We will be using this type of game as a structure for the LLM to generate games.

### Generating Features

The first step in generating a sequential game is to generate the features that will be used to represent the game. These features will generated by the LLM to generate the game.

In [23]:
def generate_game(prompt):
    prior = '''
    Your job will be to look for a fun way to combine real-world objects to make a sequential game (game theory). Your response should be formatted as:

    Representative Emoji: eg. 🎲

    Game Title: eg. Dice Roll

    Set-Up Instructions: eg.
    1. Place the dice in the center of the table.
    2. Each player takes turns rolling the dice.

    Turn-Instructions: eg.
    1. Player rolls the dice.
    2. Player reads the number at the top of the dice.

    Winning/Losing Condition: e.
    1. The player who rolls the highest number wins.
    2. The player who rolls the lowest number loses.

    Winning/Losing Condition: eg.
    1. The player who rolls the highest number wins.
    2. The player who rolls the lowest number loses.

    Ignored-Objects: eg.
    1. Coin
    2. Chess Board

    -------------------

    Your response should have NO ambiguity. You also do not need to use all objects (IGNORE IRRELEVANT SURFACES AND FURNITURE).
    '''
    response = prompt_llm(
        prior=prior,
        prompt=prompt,
    )
    return response

### Eliminating Irrelevant Features

For now, we will simply prompt the LLM to ignore objects that it thinks do not matter. In the future this can be done by a classifier.

In [6]:
objects = ["yogurt cup", "ping pong ball", "banana", "table", "sofa"]
quantities = [5, 2, 1, 1, 1]
prompt = ', '.join([f"{quant} {obj}s" if quant > 1 else obj for obj, quant in zip(objects, quantities)])
game_response = generate_game(prompt)
print(game_response)

Representative Emoji: 🍨

Game Title: Yogurt Pong

Set-Up Instructions:
1. Arrange the 5 yogurt cups in a small triangle (like bowling pins) on one end of the table.
2. Each player stands at the opposite end of the table with a ping pong ball.
3. The banana is placed beside the table as a marker for where players must stand while throwing.

Turn-Instructions: 
1. Player 1 attempts to throw their ping pong ball into one of the yogurt cups.
2. If the ball lands in a cup, the cup is removed from the table.
3. Player 2 then takes their turn, following the same rules.
4. Players take turns until all cups have been removed.

Winning/Losing Condition: 
1. The player who removes the most cups wins.
2. If both players remove an equal number of cups, the player who removed their cups in the least number of throws wins.

Ignored-Objects: 
1. Sofa


In [7]:
def numbered_list_to_array(text: str):
    return [line[line.index('.') + 2:] for line in text.split('\n') if line.strip() != ""]

In [8]:
import regex as re
structured_response = [line.split(':') for line in game_response.split("\n\n")]
# turn the response into a dictionary
meta_game = {
    key: numbered_list_to_array(value) if re.match(r'^\s*\d+\.\s+.+', value) else value.strip() 
    for key, value in structured_response
}
meta_game

{'Representative Emoji': '🍨',
 'Game Title': 'Yogurt Pong',
 'Set-Up Instructions': ['Arrange the 5 yogurt cups in a small triangle (like bowling pins) on one end of the table.',
  'Each player stands at the opposite end of the table with a ping pong ball.',
  'The banana is placed beside the table as a marker for where players must stand while throwing.'],
 'Turn-Instructions': ['Player 1 attempts to throw their ping pong ball into one of the yogurt cups.',
  'If the ball lands in a cup, the cup is removed from the table.',
  'Player 2 then takes their turn, following the same rules.',
  'Players take turns until all cups have been removed.'],
 'Winning/Losing Condition': ['The player who removes the most cups wins.',
  'If both players remove an equal number of cups, the player who removed their cups in the least number of throws wins.'],
 'Ignored-Objects': ['Sofa']}

In [9]:
formatted_ignored_objects = [obj.lower() for obj in meta_game.get("Ignored-Objects", [])]
used_objects = [obj for obj in objects if obj not in formatted_ignored_objects]
used_objects

['yogurt cup', 'ping pong ball', 'banana', 'table']

In [10]:
actions_payoffs_resp = prompt_llm(
    "You are building a sequential game in game theory. Answer with just the actions that a player can do on their turn and their payoffs in a numbered list. Answer ONLY with the numbered list in the form: 1. Action, Payoff (eg. +3)",
    f'''Here are the objects in the game: {used_objects}. MAKE SURE YOUR ACTIONS ARE MUTUALLY INDEPENDENT''',
    contexts=[game_response],
    temp=0.3
)

print(actions_payoffs_resp)

1. Throw ping pong ball at yogurt cups, Payoff +1 for each cup hit
2. Miss yogurt cups with ping pong ball, Payoff -1
3. Move closer to the table by moving the banana marker, Payoff -2
4. Move further from the table by moving the banana marker, Payoff +2
5. Attempt to hit multiple yogurt cups with one throw, Payoff +2 for each additional cup hit
6. Fail to hit multiple yogurt cups with one throw, Payoff -2 for each additional cup missed
7. Knock over a yogurt cup without the ping pong ball landing in it, Payoff -3
8. Successfully land the ping pong ball in a yogurt cup without knocking it over, Payoff +3
9. Attempt to bounce the ping pong ball off the table into a yogurt cup, Payoff +4 if successful
10. Fail to bounce the ping pong ball off the table into a yogurt cup, Payoff -4 if unsuccessful.


In [11]:
actions_payoffs = [action_payoff.split(", ") for action_payoff in numbered_list_to_array(actions_payoffs_resp)]
actions = [action for action, _ in actions_payoffs]

In [12]:
game_code = generate_code(
    f"Write a Python function that takes as input an array of action strings and number of players and returns the winner.",
    contexts=[game_response, f"Actions and Payoffs:\n{actions_payoffs_resp}"],
)


print(game_code)


def calculate_winner(actions, num_players):
    scores = [0]*num_players
    for i, action in enumerate(actions):
        if action == 'hit':
            scores[i%num_players] += 1
        elif action == 'miss':
            scores[i%num_players] -= 1
        elif action == 'move closer':
            scores[i%num_players] -= 2
        elif action == 'move further':
            scores[i%num_players] += 2
        elif action == 'hit multiple':
            scores[i%num_players] += 2
        elif action == 'miss multiple':
            scores[i%num_players] -= 2
        elif action == 'knock over':
            scores[i%num_players] -= 3
        elif action == 'land without knock over':
            scores[i%num_players] += 3
        elif action == 'bounce hit':
            scores[i%num_players] += 4
        elif action == 'bounce miss':
            scores[i%num_players] -= 4
    return scores.index(max(scores)) + 1



In [13]:
def extract_function_name(code):
    return code.split('(')[0].split()[-1]

run_game = extract_function_name(game_code)
print(run_game)

calculate_winner


In [14]:
game_actions = [actions[1], actions[0], actions[1]]
game_actions

['Miss yogurt cups with ping pong ball',
 'Throw ping pong ball at yogurt cups',
 'Miss yogurt cups with ping pong ball']

In [15]:
exec(game_code)

exec(f"winner = {run_game}({game_actions}, 2)")
print(winner)

1


## Game Generation

With all the pieces in place, we can now make a function to generate a game given a set of objects.

In [26]:
def generate_game_function(objects: list[str], quantities: list[int]) -> str:
    '''
    @param objects: list of objects to be used in the game
    @return: string representing the python function to run the game
    '''
    if len(objects) != len(quantities):
        raise ValueError("The number of objects and quantities must be the same")
    prompt = ', '.join([f"{quant} {obj}s" if quant > 1 else obj for obj, quant in zip(objects, quantities)])
    game_response = generate_game(prompt)
    print(game_response)
    structured_response = [line.split(':') for line in game_response.split("\n\n")]
    # turn the response into a dictionary
    meta_game = {
        key: numbered_list_to_array(value) if re.match(r'^\s*\d+\.\s+.+', value) else value.strip() 
        for key, value in structured_response
    }
    formatted_ignored_objects = [obj.lower() for obj in meta_game.get("Ignored-Objects", [])]
    used_objects = [obj for obj in objects if obj not in formatted_ignored_objects]
    actions_payoffs_resp = prompt_llm(
        "You are building a sequential game in game theory. Answer with just the actions that a player can do on their turn and their payoffs in a numbered list. Answer ONLY with the numbered list in the form: 1. Action, Payoff (eg. +3)",
        f'''Here are the objects in the game: {used_objects}. MAKE SURE YOUR ACTIONS ARE MUTUALLY INDEPENDENT''',
        contexts=[game_response],
        temp=0.3
    )
    actions_payoffs = [action_payoff.split(", ") for action_payoff in numbered_list_to_array(actions_payoffs_resp)]
    actions = [action for action, _ in actions_payoffs]
    game_code = generate_code(
        f"Write a Python function that takes as input an array of action strings and number of players and returns the winner.",
        contexts=[game_response, f"Actions and Payoffs:\n{actions_payoffs_resp}"],
    )
    return game_code

In [27]:
generate_game_function(objects=["charging cable", "eggs", "laptop", "basket", "keyboard"], quantities=[1, 12, 1, 3, 1])

Representative Emoji: 🥚

Game Title: Egg Transfer

Set-Up Instructions:
1. Place the 3 baskets in a line, 5 feet apart from each other.
2. Place the 12 eggs in the first basket.
3. Each player positions themselves at the first basket.

Turn-Instructions:
1. On their turn, a player must use the charging cable to lift an egg from the first basket.
2. The player must then walk to the second basket and place the egg in it without dropping it.
3. If the player successfully places the egg in the second basket, they must then use the charging cable to transfer the egg from the second basket to the third basket.
4. If a player drops an egg at any point, their turn ends.

Winning/Losing Condition:
1. The player who transfers the most eggs from the first basket to the third basket in a set time limit (e.g. 5 minutes) wins.
2. The player who transfers the least number of eggs or breaks the most eggs loses.

Ignored-Objects:
1. Laptop
2. Keyboard


'\ndef find_winner(actions, num_players):\n    scores = [0]*num_players\n    current_player = 0\n\n    for action in actions:\n        if action == "Pick up egg with charging cable":\n            scores[current_player] += 1\n        elif action == "Walk to second basket without dropping egg":\n            scores[current_player] += 2\n        elif action == "Place egg in second basket":\n            scores[current_player] += 1\n        elif action == "Pick up egg from second basket with charging cable":\n            scores[current_player] += 1\n        elif action == "Walk to third basket without dropping egg":\n            scores[current_player] += 2\n        elif action == "Place egg in third basket":\n            scores[current_player] += 3\n        elif action == "Drop egg at any point":\n            scores[current_player] -= 2\n\n        current_player = (current_player + 1) % num_players\n\n    max_score = max(scores)\n    winners = [i+1 for i, score in enumerate(scores) if score 